# ここでやること
<br>

1. 遺伝アルゴリズムダウンサンプリングの実装
    1. 評価関数の作成
    2. 個体の定義
    3. 交叉の定義
    4. 変異の定義
    5. 選択の設定
    
2. パラメータの決定
    1. よく変更するパラメータ：　交叉確率、変異確率、選択特徴ベクトルサイズ
    2. あまり変更しないパラメータ：　世代数、集団人口
    
3. ダウンサンプリングの実行
    1. 点群データセットの読み込み←事前準備
    2. ベース特徴ベクトルの読み込み(2468, 1024)←事前準備
    3. classification_first_modelの読み込み
    4. 〜計算〜
    5. インデックスを保存(2468, 1024)
    
4. 評価→後々移動
    1. 点群データセットの読み込み
    2. ダウンサンプリング後インデックスの読み込み
    3. classification_second_modelの読み込み
    4. 〜計算〜
    5. ランダムサンプリングとの精度比較
    6. 最遠点サンプリングとの精度比較

In [1]:
import random
import pandas as pd
import numpy as np
# import datetime

from copy import deepcopy

from pathlib import Path
import sys
import importlib

from easydict import EasyDict
from pprint import pprint
import torch
import os
import time

from deap import creator, base, tools, algorithms
# import argparse

import time
from tqdm import tqdm
import datetime

In [2]:
#Slackで計算終了などを通知してくれるもの
#自分のwebhook_urlを設定
# import requests
# import json

# def slack_notify(message):
#     webhook_url = ''

#     text = message

#     requests.post(webhook_url, data = json.dumps({
#         "text": text
#     }));
    
# slack_notify('test')

In [2]:
#ランダム値をシード値で初期化する関数
def initialize(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [3]:
BASE_DIR = Path('../Pointnet_Pointnet2_pytorch')
sys.path.append(str(BASE_DIR))
sys.path.append(str(BASE_DIR/'models'))
print(sys.path)

from data_utils.ModelNetDataLoader import ModelNetDataLoader

RESULT_DIR = BASE_DIR/'../Mywork/Results'
FIGURE_DIR = RESULT_DIR/'Figures'
FILE_DIR = RESULT_DIR/'Files'

['/mnt/workspace/Github/Share/Mywork', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/tmp/OpenPCDet', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '../Pointnet_Pointnet2_pytorch', '../Pointnet_Pointnet2_pytorch/models']


In [4]:
#GA_downsampling用のargs
args = EasyDict({
    'use_cpu': 'False',
    'gpu': '0',
    'batch_size': 103, 
    'num_category': 40,
    'num_point': 1024,
    'log_dir': 'pointnet2_cls_ssg',
    'use_normals': False, #bestを使うときはTrue
    'use_uniform_sample': False, #Trueにするとデータロード時にFPS実行。FPSには初期値にランダム性あり。TrueとFalseが逆な気がする
    'num_votes': 3,
    'output': FILE_DIR,
    })

num_class = args.num_category
print("The num of class is {0}".format(num_class))
print(args.log_dir)

The num of class is 40
pointnet2_cls_ssg


### データセットの用意

In [5]:
print(os.listdir(FILE_DIR/'seed0'/'Feature_vector_base'))

['average2021-12-16_22-54.csv', '2021-12-04_17-59.csv', '2021-12-07_13-27.csv', '2021-12-04_17-51.csv']


In [6]:
import DataLoaderForGAdownsampling

data_path = BASE_DIR / Path('data/modelnet40_normal_resampled/')
fv_path = FILE_DIR/'seed0'/'Feature_vector_base'/'2021-12-07_13-27.csv'
# fv_path = FILE_DIR/'seed0'/'Feature_vector_base'/'average2021-12-16_22-54.csv'

GA_dataset = DataLoaderForGAdownsampling.FVDataLoader(root=data_path, fv_path=fv_path, args=args, split='test', process_data=False)
GADataLoader = torch.utils.data.DataLoader(GA_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

The size of test data is 2468


### 分類モデル用意（点群 -> 特徴ベクトル、特徴ベクトル -> ラベル）

In [7]:
model_name = "cls_ssg_feature_selection" #オリジナルのcls_ssgを２つに分解。特徴抽出部分がfirst、分類部分がsecond
model = importlib.import_module(model_name)

Feature_vector_calculator = model.get_first_model(num_class, normal_channel=args.use_normals)
Feature_vector_calculator = Feature_vector_calculator.cuda()
Classifier_from_feature_vector = model.get_second_model(num_class, normal_channel=args.use_normals)

experiment_dir = BASE_DIR / Path('log/classification/') / Path(args.log_dir)
checkpoint = torch.load(str(experiment_dir) + '/checkpoints/best_model.pth') #学習済みのチェックポイントを読み込み

Feature_vector_calculator.load_state_dict(checkpoint['model_state_dict'])
Classifier_from_feature_vector.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

# 遺伝ダウンサンプリング

In [8]:
def get_fitness(individual, points, target_feature_vector, target, model1, model2, iteration):
    score = torch.empty(0).cuda()
    Feature_vector_calculator = model1.eval()
    Classifier_from_feature_vector = model2.eval()
    B, N, _ = points.shape
    points, target = points.cuda(), target.cuda()
    target_feature_vector = target_feature_vector.to(torch.float32).cuda()
    points = points[:, individual, :] #選択インデックスの点群を削除 1024->512
    points = torch.tile(points, (1, int(N/len(individual)), 1)) #アップサンプリングしてMLに入力 512->1024
    points = points.transpose(2, 1)
        
    with torch.no_grad():
        for i in range(iteration):        
            feature_vector = Feature_vector_calculator(points)
            dist = torch.sum(torch.cdist(feature_vector.view(B, 1, -1), target_feature_vector.view(B, 1, -1)), dim=0)
            score = torch.cat((score, dist[0]), 0)
    score = torch.mean(score).to('cpu').detach().numpy().copy()
    return score,

def Accuracy(individual, points, target_feature_vector, target, model1, model2, iteration):
    mean_correct = []
    Feature_vector_calculator = model1.eval()
    Classifier_from_feature_vector = model2.eval()
    B, N, _ = points.shape
#     print(type(points))
    points, target = points.cuda(), target.cuda()
    target_feature_vector = target_feature_vector.to(torch.float32).cuda()
    points = points[:, individual, :] #選択インデックスの点群を削除 1024->512
    points = torch.tile(points, (1, int(N/len(individual)), 1)) #アップサンプリングしてMLに入力 512->1024
    points = points.transpose(2, 1)
        
    with torch.no_grad():
        for i in range(iteration):        
            feature_vector = Feature_vector_calculator(points)
            pred, _ = Classifier_from_feature_vector(feature_vector)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.long().data).cpu().sum()
            mean_correct.append(correct.item() / float(points.size()[0]))
    instance_acc = np.mean(mean_correct)
    return instance_acc,

#初期のインデックス作成
def create_select_idx(idx_size, select_size):
    select_idx = random.sample(range(idx_size), select_size)

    return select_idx

#突然変異
#k点入れ替え
def mutation(individual, idx_size, select_size, k):
    remain_idx = random.sample(individual, (select_size - k))
    excepted_set = set(range(idx_size)) - set(individual)
    join_idx = random.sample(list(excepted_set), k)
    individual[:] = (remain_idx + join_idx)

    return individual,

def cxDifIdx(ind1, ind2):
    ind_tmp1 = set(ind1)
    ind_tmp2 = set(ind2)

    com = ind_tmp1 & ind_tmp2
    
    ind_tmp1 = list(ind_tmp1 - com)
#     ind_tmp1.sort()
    ind1[:] = ind_tmp1
    ind_tmp2 = list(ind_tmp2 - com)
#     ind_tmp2.sort()
    ind2[:] = ind_tmp2
    
    if len(ind1) > 1:

        ind1, ind2 = tools.cxTwoPoint(ind1, ind2)

    ind1 += com
    ind2 += com

    ind1.sort()
    ind2.sort()
    
    return ind1, ind2

In [9]:
def run_GA(original_points, target_feature_vector, target, model1, model2, sampling_rate, ngen, seed, cxpd, mutpd):
    npoints = 1024
    select_size = int(npoints * sampling_rate)
    iteration = 10 #評価の際に特徴ベクトルを計算する回数
    initialize(seed)
    accuracy = np.empty(0)
    
    x = original_points
    
    y1 = target_feature_vector
    y2 = target

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    toolbox.register("select_idx", create_select_idx, npoints, select_size)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.select_idx)

    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("evaluate", get_fitness, points=x, target_feature_vector=y1, target=y2, model1=model1, \
                     model2=model2, iteration=iteration)
    
    toolbox.register("mate", cxDifIdx)

    toolbox.register("mutate", mutation, idx_size=npoints, select_size=select_size, k=1)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    toolbox.register("Accuracy", Accuracy, points=x, target_feature_vector=y1, target=y2, model1=model1, \
                     model2=model2, iteration=iteration)

    # 世代数の設定
    pop = toolbox.population(n=30)
    hof = tools.ParetoFront()
    stats_acc = tools.Statistics(key=toolbox.Accuracy)
    stats_dist = tools.Statistics(lambda ind: ind.fitness.values)
    stats = tools.MultiStatistics(dist=stats_dist, acc=stats_acc)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)
    
    # 実行
    start_time = time.perf_counter()
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=cxpd, mutpb=mutpd, ngen=ngen, stats=stats, halloffame=hof,verbose=True)
    stop_time = time.perf_counter()

    elapsed_time = stop_time - start_time
    print("処理時間：", elapsed_time)
    
    return hof, pop, log

In [10]:
def make_path(*a):
    path = '/'.join(a)
    
    return path

def make_prefix(**a):
    prefix = ''
    for key in a.keys():
        prefix += key + '{}_'
    prefix = prefix.format(*tuple(a.values()))
    return prefix

def separate_array(object_arr):
    size = object_arr.shape[0]
    new_arr = np.empty(0)
    for i in range(size):
        new_arr = np.append(new_arr, object_arr[i][0])
        new_arr = np.append(new_arr, object_arr[i][1])
    new_arr = new_arr.reshape(size, 2)
    return new_arr

#出力結果を保存する関数
#statsに一つ（dist=fitness）しか登録していないバージョン
# def save_GA_results(log, hof, pop, *args, **kwargs):
#     npoints = 1024
#     hof_bit = []
#     for individual in hof:
#         ind_bit = np.zeros(npoints)

#         ind_bit[individual] = 1
#         hof_bit.append(ind_bit)

#     hof_bit = pd.DataFrame(data=hof_bit)

#     pprint(hof_bit)
    
#     method_name = make_path(*args) + '/Hall of fame' #何の結果か、どのような加工を加えているか、何個ずつか
#     prefix = make_prefix(**kwargs) #よく変えるパラメータ
#     ext = ".csv"
#     df = hof_bit
#     #-------------------------------------
#     save_dir = FILE_DIR / Path(method_name) 
#     # print(save_dir)
#     save_dir.mkdir(exist_ok=True, parents=True)
#     timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
#     # print(timestr)
#     file_name = prefix + timestr + ext
#     df.to_csv(save_dir/file_name, index=False)
#     print("Saved in:", save_dir/file_name)
#     slack_notify("Saved in:"+ str(save_dir/file_name))


#     #logを保存できる形に変換
#     log_df = pd.DataFrame(data=log)
#     add_columns = ["dist avg", "acc avg", "dist std", "acc std", "dist min", "acc min", "dist max", "acc max"]
#     arr = np.empty(0)
#     for i in range(4):
#         arr = np.append(arr, separate_array(log_df.iloc[:, i+2].values))
#     arr = arr.reshape(len(log_df), len(add_columns))
#     arr = arr.T
# #     arr[1::2] = arr[1::2] / 1e-15
#     for i, column in enumerate(add_columns):
#         log_df[column] = arr[i]
#     drop_columns = ['avg', 'std', 'min', 'max']
#     log_df = log_df.drop(drop_columns, axis=1)
#     log_df.iloc[:, 2:] = log_df.iloc[:, 2:].astype("float32")

#     method_name = make_path(*args) + '/log' #何の結果か、どのような加工を加えているか、何個ずつか
#     prefix = make_prefix(**kwargs)
#     ext = ".csv"
#     df = log_df
#     #-------------------------------------
#     save_dir = FILE_DIR / Path(method_name) 
#     # print(save_dir)
#     save_dir.mkdir(exist_ok=True, parents=True)
#     timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
#     # print(timestr)
#     file_name = prefix + timestr + ext
#     df.to_csv(save_dir/file_name, index=False)
#     print("Saved in:", save_dir/file_name)
#     slack_notify("Saved in:"+ str(save_dir/file_name))
    
#     #最終populationを保存
#     pop_df = pd.DataFrame(data=pop)
#     method_name = make_path(*args) + '/pop' #何の結果か、どのような加工を加えているか、何個ずつか
#     prefix = make_prefix(**kwargs)
#     ext = ".csv"
#     df = pop_df
#     #-------------------------------------
#     save_dir = FILE_DIR / Path(method_name) 
#     # print(save_dir)
#     save_dir.mkdir(exist_ok=True, parents=True)
#     timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
#     # print(timestr)
#     file_name = prefix + timestr + ext
#     df.to_csv(save_dir/file_name, index=False)
#     print("Saved in:", save_dir/file_name)
#     slack_notify("Saved in:"+ str(save_dir/file_name))
    
#     return None

In [14]:
#出力結果を保存する関数
#statsに２つ登録しているバージョン。dist, acc
def save_GA_results(log, hof, pop, *args, **kwargs):
    npoints = 1024
#hofの保存    
    hof_bit = []
    for individual in hof:
        ind_bit = np.zeros(npoints)

        ind_bit[individual] = 1
        hof_bit.append(ind_bit)

    hof_bit = pd.DataFrame(data=hof_bit)

    pprint(hof_bit)
    
    method_name = make_path(*args) + '/Hall of fame' #何の結果か、どのような加工を加えているか、何個ずつか
    prefix = make_prefix(**kwargs) #よく変えるパラメータ
    ext = ".csv"
    df = hof_bit
    #-------------------------------------
    save_dir = FILE_DIR / Path(method_name) 
    # print(save_dir)
    save_dir.mkdir(exist_ok=True, parents=True)
    timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    # print(timestr)
    file_name = prefix + timestr + ext
    df.to_csv(save_dir/file_name, index=False)
    print("Saved in:", save_dir/file_name)
#     slack_notify("Saved in:"+ str(save_dir/file_name))
    
#logの保存
    log_df1 = pd.DataFrame(log.chapters['dist']) 
    log_df1 = log_df1.drop(['nevals'], axis=1)
    log_df1 = log_df1.rename(columns={'avg':'dist avg', 'std':'dist std', 'min':'dist min', 'max':'dist max'})
    log_df2 = pd.DataFrame(log.chapters['acc'])
    log_df2 = log_df2.drop(['nevals'], axis=1)
    log_df2 = log_df2.rename(columns={'avg':'acc avg', 'std':'acc std', 'min':'acc min', 'max':'acc max'})
    log_df = pd.merge(log_df1, log_df2, on='gen')
    # columns = log_df.columns.values
    # columns = np.insert(columns, 0, columns[4])
    # columns = np.delete(columns, 5)
    log_df = log_df.drop(['gen'], axis=1)
    columns = ["dist avg", "acc avg", "dist std", "acc std", "dist min", "acc min", "dist max", "acc max"]
    log_df = log_df.reindex(columns=columns)
    log_df = log_df.astype("float32")

    method_name = make_path(*args) + '/log' #何の結果か、どのような加工を加えているか、何個ずつか
    prefix = make_prefix(**kwargs)
    ext = ".csv"
    df = log_df
    #-------------------------------------
    save_dir = FILE_DIR / Path(method_name) 
    # print(save_dir)
    save_dir.mkdir(exist_ok=True, parents=True)
    timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    # print(timestr)
    file_name = prefix + timestr + ext
    df.to_csv(save_dir/file_name, index=False)
    print("Saved in:", save_dir/file_name)
#     slack_notify("Saved in:"+ str(save_dir/file_name))
    
    #最終populationを保存
#     pop = np.array(pop)
    pop_df = pd.DataFrame(data=pop)
    method_name = make_path(*args) + '/pop'
    prefix = make_prefix(**kwargs) 
    ext = ".csv"
    df = pop_df
    #-------------------------------------
    save_dir = FILE_DIR / Path(method_name) 
    # print(save_dir)
    save_dir.mkdir(exist_ok=True, parents=True)
    timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    # print(timestr)
    file_name = prefix + timestr + ext
    df.to_csv(save_dir/file_name, index=False)
    print("Saved in:", save_dir/file_name)
#     slack_notify("Saved in:"+ str(save_dir/file_name))
    
    return None

# ここからデータ取得

In [12]:
#テストに使うデータのインデックスを指定（0~2467)
data_idx = 615
original_points, target_fv, target = GA_dataset.__getitem__(data_idx)
#モデルに入力するためにtorchのtensor型に変換
original_points = torch.from_numpy(original_points.astype(np.float32)).clone().view(1, 1024, 3)
target_fv = torch.from_numpy(target_fv.astype(np.float32)).clone().view(1, -1)
target = torch.from_numpy(np.array(target).astype(np.float32)).clone()

Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cuda()
Feature_vector_calculator = Feature_vector_calculator.eval().cuda()

rate = 1/2
ngen = 1 #テストとして小さい値を使う。修論では1000
seed = 0
cxpd = 0.2
mutpd = 0.5

hof, pop, log= run_GA(original_points, target_fv, target, Feature_vector_calculator, \
                      Classifier_from_feature_vector, rate, ngen, seed, cxpd, mutpd) #ここで計算実行
# save_GA_results(log, hof, pop, 'seed0', 'GA_downsampling',data=data_idx, ngen=ngen, rate=rate, seed=seed)#保存

   	      	                            acc                             	                                    dist                                    
   	      	------------------------------------------------------------	----------------------------------------------------------------------------
gen	nevals	avg         	gen	max 	min  	nevals	std         	avg         	gen	max         	min         	nevals	std         
0  	30    	[0.93333333]	0  	[1.]	[0.2]	30    	[0.18678568]	[3.19589295]	0  	[4.04351044]	[2.70931792]	30    	[0.35561944]
1  	15    	[0.99]      	1  	[1.]	[0.9]	15    	[0.03]      	[2.99299343]	1  	[3.38764811]	[2.62482691]	15    	[0.20202203]
処理時間： 113.44768102373928


In [15]:
save_GA_results(log, hof, pop, 'seed0', 'GA_downsampling', 'test', data=data_idx, ngen=ngen, rate=rate, seed=seed)

   0     1     2     3     4     5     6     7     8     9     ...  1014  \
0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0  ...   0.0   

   1015  1016  1017  1018  1019  1020  1021  1022  1023  
0   0.0   1.0   1.0   0.0   1.0   0.0   0.0   1.0   1.0  

[1 rows x 1024 columns]
Saved in: ../Pointnet_Pointnet2_pytorch/../Mywork/Results/Files/seed0/GA_downsampling/test/Hall of fame/data615_ngen1_rate0.5_seed0_2022-02-24_10-46.csv
Saved in: ../Pointnet_Pointnet2_pytorch/../Mywork/Results/Files/seed0/GA_downsampling/test/log/data615_ngen1_rate0.5_seed0_2022-02-24_10-46.csv
Saved in: ../Pointnet_Pointnet2_pytorch/../Mywork/Results/Files/seed0/GA_downsampling/test/pop/data615_ngen1_rate0.5_seed0_2022-02-24_10-46.csv


In [233]:
data_idx = 615
original_points, target_fv, target = GA_dataset.__getitem__(data_idx)
original_points = torch.from_numpy(original_points.astype(np.float32)).clone().view(1, 1024, 3)
target_fv = torch.from_numpy(target_fv.astype(np.float32)).clone().view(1, -1)
target = torch.from_numpy(np.array(target).astype(np.float32)).clone()

Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cuda()
Feature_vector_calculator = Feature_vector_calculator.eval().cuda()

rate = 1/4
ngen = 1000
seed = 0
cxpd = 0.2
mutpd = 0.5

hof, pop, log= run_GA(original_points, target_fv, target, Feature_vector_calculator, \
                      Classifier_from_feature_vector, rate, ngen, seed, cxpd, mutpd)
save_GA_results(log, hof, pop, 'seed0', 'GA_downsampling',data=data_idx, ngen=ngen, rate=rate, seed=seed)

   	      	                            acc                             	                                    dist                                    
   	      	------------------------------------------------------------	----------------------------------------------------------------------------
gen	nevals	avg         	gen	max 	min 	nevals	std         	avg         	gen	max         	min         	nevals	std         
0  	30    	[0.58666667]	0  	[1.]	[0.]	30    	[0.47027179]	[4.97807967]	0  	[5.82215643]	[4.22988081]	30    	[0.43114419]
1  	23    	[0.59666667]	1  	[1.]	[0.]	23    	[0.47151057]	[4.55913908]	1  	[5.39813995]	[3.98905993]	23    	[0.31960771]
2  	20    	[0.49666667]	2  	[1.]	[0.]	20    	[0.46510453]	[4.40763302]	2  	[5.0579195] 	[4.06211519]	20    	[0.25130353]
3  	15    	[0.4]       	3  	[1.]	[0.]	15    	[0.48989795]	[4.19501808]	3  	[4.43740797]	[4.03622007]	15    	[0.13082222]
4  	15    	[0.3]       	4  	[1.]	[0.]	15    	[0.45825757]	[4.11552003]	4  	[4.36844921]	[4.018579

64 	18    	[0.98666667]	64 	[1.] 	[0.6]	18    	[0.0718022] 	[2.9145398] 	64 	[3.12183142]	[2.8552444] 	18    	[0.06140652]
65 	16    	[1.]        	65 	[1.] 	[1.] 	16    	[0.]        	[2.88989666]	65 	[2.99289131]	[2.84405589]	16    	[0.04176578]
66 	20    	[0.99]      	66 	[1.] 	[0.7]	20    	[0.05385165]	[2.88672756]	66 	[3.03486753]	[2.78581357]	20    	[0.05886124]
67 	19    	[1.]        	67 	[1.] 	[1.] 	19    	[0.]        	[2.88790674]	67 	[3.07307363]	[2.8222425] 	19    	[0.0523171] 
68 	17    	[0.97]      	68 	[1.] 	[0.5]	17    	[0.09363048]	[2.88714166]	68 	[3.08128762]	[2.8222425] 	17    	[0.06313405]
69 	18    	[0.99666667]	69 	[1.] 	[0.9]	18    	[0.01795055]	[2.87910879]	69 	[3.1529026] 	[2.81973052]	18    	[0.06912276]
70 	25    	[0.96333333]	70 	[1.] 	[0.4]	25    	[0.11685699]	[2.86954404]	70 	[3.00564384]	[2.78770614]	25    	[0.05394168]
71 	19    	[0.98333333]	71 	[1.] 	[0.7]	19    	[0.05821416]	[2.87100285]	71 	[3.09818935]	[2.78770614]	19    	[0.07134338]
72 	16    	[0.97

131	17    	[1.]        	131	[1.] 	[1.] 	17    	[0.]        	[2.59055414]	131	[2.79731178]	[2.53595042]	17    	[0.05426496]
132	19    	[1.]        	132	[1.] 	[1.] 	19    	[0.]        	[2.59861977]	132	[2.77175903]	[2.52213669]	19    	[0.05478017]
133	14    	[1.]        	133	[1.] 	[1.] 	14    	[0.]        	[2.58183334]	133	[2.72936177]	[2.52213669]	14    	[0.05355333]
134	17    	[1.]        	134	[1.] 	[1.] 	17    	[0.]        	[2.59385986]	134	[2.8407197] 	[2.52213669]	17    	[0.0726087] 
135	20    	[1.]        	135	[1.] 	[1.] 	20    	[0.]        	[2.58200765]	135	[2.75493312]	[2.52213669]	20    	[0.04962747]
136	16    	[1.]        	136	[1.] 	[1.] 	16    	[0.]        	[2.58336931]	136	[2.69458556]	[2.52213669]	16    	[0.04324732]
137	17    	[1.]        	137	[1.] 	[1.] 	17    	[0.]        	[2.57720416]	137	[2.71902871]	[2.52213669]	17    	[0.05044407]
138	13    	[1.]        	138	[1.] 	[1.] 	13    	[0.]        	[2.57422062]	138	[2.73370862]	[2.52213669]	13    	[0.05490862]
139	21    	[1.] 

198	19    	[1.]        	198	[1.] 	[1.] 	19    	[0.]        	[2.49494249]	198	[2.94160819]	[2.39792895]	19    	[0.09916812]
199	17    	[0.99333333]	199	[1.] 	[0.8]	17    	[0.0359011] 	[2.47203606]	199	[2.67086577]	[2.39792895]	17    	[0.05855603]
200	19    	[0.99333333]	200	[1.] 	[0.8]	19    	[0.0359011] 	[2.48253152]	200	[2.9569478] 	[2.41255116]	19    	[0.10045158]
201	15    	[0.99666667]	201	[1.] 	[0.9]	15    	[0.01795055]	[2.47485655]	201	[2.63973927]	[2.41922736]	15    	[0.05099248]
202	17    	[1.]        	202	[1.] 	[1.] 	17    	[0.]        	[2.47563237]	202	[2.58664966]	[2.41922736]	17    	[0.04695893]
203	19    	[0.99333333]	203	[1.] 	[0.8]	19    	[0.0359011] 	[2.46368659]	203	[2.56748581]	[2.41922736]	19    	[0.04221156]
204	18    	[1.]        	204	[1.] 	[1.] 	18    	[0.]        	[2.47424402]	204	[2.604491]  	[2.41922736]	18    	[0.05285814]
205	21    	[1.]        	205	[1.] 	[1.] 	21    	[0.]        	[2.49442927]	205	[2.63892412]	[2.41922736]	21    	[0.0609392] 
206	26    	[0.99

265	17    	[0.98333333]	265	[1.] 	[0.8]	17    	[0.04533824]	[2.33355205]	265	[2.54444504]	[2.25480151]	17    	[0.06975038]
266	22    	[0.96]      	266	[1.] 	[0.4]	22    	[0.11430952]	[2.34911925]	266	[2.62558365]	[2.26059794]	22    	[0.0751633] 
267	18    	[0.94333333]	267	[1.] 	[0.6]	18    	[0.09551033]	[2.3128274] 	267	[2.44531703]	[2.2511375] 	18    	[0.04074451]
268	16    	[0.94666667]	268	[1.] 	[0.6]	16    	[0.09910712]	[2.32632103]	268	[2.46020198]	[2.2511375] 	16    	[0.0492741] 
269	21    	[0.89666667]	269	[1.] 	[0.2]	21    	[0.17603661]	[2.34548123]	269	[2.6579895] 	[2.27486777]	21    	[0.08306273]
270	22    	[0.90333333]	270	[1.] 	[0.4]	22    	[0.14019827]	[2.38229139]	270	[2.79358363]	[2.27677321]	22    	[0.12267826]
271	16    	[0.93333333]	271	[1.] 	[0.3]	16    	[0.13743685]	[2.3377706] 	271	[2.4935894] 	[2.27677321]	16    	[0.0522725] 
272	17    	[0.93333333]	272	[1.] 	[0.5]	17    	[0.12202003]	[2.34612177]	272	[2.52228022]	[2.28267932]	17    	[0.06147275]
273	16    	[0.93

...

734	17    	[0.99333333]	734	[1.] 	[0.9]	17    	[0.02494438]	[2.06492215]	734	[2.26141024]	[1.97137833]	17    	[0.07569101]
735	14    	[0.99333333]	735	[1.] 	[0.8]	14    	[0.0359011] 	[2.07977653]	735	[2.48864675]	[1.97137833]	14    	[0.11469788]
736	16    	[0.96333333]	736	[1.] 	[0.3]	16    	[0.13034143]	[2.06596074]	736	[2.20988441]	[1.97137833]	16    	[0.07875766]
737	19    	[0.99666667]	737	[1.] 	[0.9]	19    	[0.01795055]	[2.08157225]	737	[2.24600005]	[1.97137833]	19    	[0.07461286]
738	14    	[0.98]      	738	[1.] 	[0.8]	14    	[0.04760952]	[2.06882598]	738	[2.39548278]	[1.97137833]	14    	[0.09845056]
739	15    	[0.95333333]	739	[1.] 	[0.3]	15    	[0.14996296]	[2.06420383]	739	[2.26865458]	[1.97137833]	15    	[0.08068924]
740	23    	[0.97]      	740	[1.] 	[0.3]	23    	[0.12688578]	[2.08344684]	740	[2.25412965]	[1.9824971] 	23    	[0.07688574]
741	22    	[1.]        	741	[1.] 	[1.] 	22    	[0.]        	[2.07283196]	741	[2.27472854]	[1.9824971] 	22    	[0.0750059] 
742	19    	[1.] 

801	20    	[0.97666667]	801	[1.] 	[0.6]	20    	[0.07608475]	[2.11062686]	801	[2.49965525]	[1.99301457]	20    	[0.10746082]
802	20    	[0.97666667]	802	[1.] 	[0.8]	20    	[0.05587685]	[2.10877921]	802	[2.35202265]	[1.99301457]	20    	[0.0994144] 
803	16    	[0.99]      	803	[1.] 	[0.7]	16    	[0.05385165]	[2.0802658] 	803	[2.25707388]	[1.99301457]	16    	[0.08026093]
804	18    	[0.99]      	804	[1.] 	[0.8]	18    	[0.03958114]	[2.08837681]	804	[2.37049341]	[1.9552263] 	18    	[0.10470556]
805	22    	[0.98]      	805	[1.] 	[0.4]	22    	[0.1077033] 	[2.07821516]	805	[2.21522975]	[1.9552263] 	22    	[0.06854731]
806	25    	[0.96666667]	806	[1.] 	[0.1]	25    	[0.16193277]	[2.10625671]	806	[2.27365851]	[1.9552263] 	25    	[0.07922123]
807	17    	[0.99666667]	807	[1.] 	[0.9]	17    	[0.01795055]	[2.08810681]	807	[2.21028137]	[1.9552263] 	17    	[0.05985272]
808	18    	[0.98]      	808	[1.] 	[0.4]	18    	[0.1077033] 	[2.09938323]	808	[2.42294431]	[1.9552263] 	18    	[0.09344455]
809	14    	[1.] 

868	16    	[0.97]      	868	[1.] 	[0.1]	16    	[0.16155494]	[2.06590352]	868	[2.56058097]	[1.93498552]	16    	[0.15591718]
869	17    	[0.96333333]	869	[1.] 	[0.] 	17    	[0.17978382]	[2.03313942]	869	[2.32847071]	[1.93498552]	17    	[0.09626196]
870	12    	[0.99333333]	870	[1.] 	[0.8]	12    	[0.0359011] 	[2.03399091]	870	[2.25677872]	[1.93498552]	12    	[0.09665252]
871	14    	[0.98666667]	871	[1.] 	[0.6]	14    	[0.0718022] 	[2.03881036]	871	[2.29763675]	[1.93498552]	14    	[0.10583958]
872	16    	[1.]        	872	[1.] 	[1.] 	16    	[0.]        	[2.02466085]	872	[2.21774936]	[1.93498552]	16    	[0.08850857]
873	24    	[1.]        	873	[1.] 	[1.] 	24    	[0.]        	[2.06907841]	873	[2.34979129]	[1.93498552]	24    	[0.11271362]
874	15    	[1.]        	874	[1.] 	[1.] 	15    	[0.]        	[2.05103418]	874	[2.30321789]	[1.93498552]	15    	[0.10155026]
875	18    	[1.]        	875	[1.] 	[1.] 	18    	[0.]        	[2.08984337]	875	[2.36874557]	[1.93498552]	18    	[0.10711184]
876	19    	[1.] 

935	21    	[0.98333333]	935	[1.] 	[0.6]	21    	[0.07340905]	[2.08418441]	935	[2.61473513]	[1.96603394]	21    	[0.12070249]
936	19    	[1.]        	936	[1.] 	[1.] 	19    	[0.]        	[2.06080454]	936	[2.21188045]	[1.96603394]	19    	[0.06998482]
937	16    	[1.]        	937	[1.] 	[1.] 	16    	[0.]        	[2.04193853]	937	[2.2427547] 	[1.96603394]	16    	[0.07990291]
938	20    	[0.99333333]	938	[1.] 	[0.8]	20    	[0.0359011] 	[2.07034368]	938	[2.21025753]	[1.96603394]	20    	[0.08571623]
939	18    	[0.98666667]	939	[1.] 	[0.6]	18    	[0.0718022] 	[2.0881276] 	939	[2.29606795]	[1.96469975]	18    	[0.10148252]
940	15    	[1.]        	940	[1.] 	[1.] 	15    	[0.]        	[2.05029079]	940	[2.21985459]	[1.96469975]	15    	[0.0835584] 
941	14    	[1.]        	941	[1.] 	[1.] 	14    	[0.]        	[2.0316805] 	941	[2.35599494]	[1.96469975]	14    	[0.09036616]
942	16    	[0.99333333]	942	[1.] 	[0.8]	16    	[0.0359011] 	[2.03923922]	942	[2.29584241]	[1.96469975]	16    	[0.09045161]
943	19    	[0.99

Saved in: /mnt/workspace/Github/Pointnet_Pointnet2_pytorch/Mywork/Results/Files/seed0/GA_downsampling/log/data615_ngen1000_rate0.25_seed0_2022-01-03_04-49.csv
Saved in: /mnt/workspace/Github/Pointnet_Pointnet2_pytorch/Mywork/Results/Files/seed0/GA_downsampling/pop/data615_ngen1000_rate0.25_seed0_2022-01-03_04-49.csv


In [234]:
data_idx = 615
original_points, target_fv, target = GA_dataset.__getitem__(data_idx)
original_points = torch.from_numpy(original_points.astype(np.float32)).clone().view(1, 1024, 3)
target_fv = torch.from_numpy(target_fv.astype(np.float32)).clone().view(1, -1)
target = torch.from_numpy(np.array(target).astype(np.float32)).clone()

Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cuda()
Feature_vector_calculator = Feature_vector_calculator.eval().cuda()

rate = 1/8
ngen = 1000
seed = 0
cxpd = 0.2
mutpd = 0.5

hof, pop, log= run_GA(original_points, target_fv, target, Feature_vector_calculator, \
                      Classifier_from_feature_vector, rate, ngen, seed, cxpd, mutpd)
save_GA_results(log, hof, pop, 'seed0', 'GA_downsampling',data=data_idx, ngen=ngen, rate=rate, seed=seed)

   	      	                     acc                     	                                    dist                                    
   	      	---------------------------------------------	----------------------------------------------------------------------------
gen	nevals	avg  	gen	max 	min 	nevals	std  	avg         	gen	max         	min         	nevals	std         
0  	30    	[0.2]	0  	[1.]	[0.]	30    	[0.4]	[6.70239379]	0  	[7.63652372]	[5.86659956]	30    	[0.41988789]
1  	22    	[0.13333333]	1  	[1.]	[0.]	22    	[0.33993463]	[6.44650429]	1  	[7.17364073]	[5.75669289]	22    	[0.35131065]
2  	21    	[0.23333333]	2  	[1.]	[0.]	21    	[0.42295258]	[6.20989599]	2  	[6.72519064]	[5.75669289]	21    	[0.25295231]
3  	18    	[0.26666667]	3  	[1.]	[0.]	18    	[0.44221664]	[6.03123091]	3  	[6.55675125]	[5.75669289]	18    	[0.19190698]
4  	21    	[0.5]       	4  	[1.]	[0.]	21    	[0.5]       	[5.98454318]	4  	[6.76727057]	[5.69057941]	21    	[0.27070873]
5  	21    	[0.63333333]	5  	[1.]	[

65 	18    	[1.]        	65 	[1.]	[1.]	18    	[0.]        	[4.44315685]	65 	[4.767663]  	[4.34733677]	18    	[0.07915346]
66 	16    	[0.93333333]	66 	[1.]	[0.]	16    	[0.24944383]	[4.44160773]	66 	[4.64002371]	[4.30129194]	16    	[0.09093621]
67 	16    	[0.96666667]	67 	[1.]	[0.]	16    	[0.17950549]	[4.42719032]	67 	[4.58935118]	[4.34733677]	16    	[0.07749112]
68 	19    	[1.]        	68 	[1.]	[1.]	19    	[0.]        	[4.41806156]	68 	[4.61617851]	[4.34612608]	19    	[0.07675114]
69 	21    	[1.]        	69 	[1.]	[1.]	21    	[0.]        	[4.42769262]	69 	[4.60484171]	[4.34415388]	21    	[0.07257245]
70 	19    	[1.]        	70 	[1.]	[1.]	19    	[0.]        	[4.39293539]	70 	[4.54936743]	[4.29918385]	19    	[0.06128557]
71 	20    	[1.]        	71 	[1.]	[1.]	20    	[0.]        	[4.39296902]	71 	[4.60843229]	[4.29918385]	20    	[0.07028852]
72 	20    	[1.]        	72 	[1.]	[1.]	20    	[0.]        	[4.40557756]	72 	[4.5531168] 	[4.29918385]	20    	[0.08370436]
73 	16    	[1.]        	73 	[1.]

133	19    	[0.86666667]	133	[1.]	[0.]	19    	[0.33993463]	[3.92538613]	133	[4.2680583] 	[3.77754903]	19    	[0.12563572]
134	19    	[0.83333333]	134	[1.]	[0.]	19    	[0.372678]  	[3.89119147]	134	[4.07971525]	[3.77754903]	19    	[0.08311796]
135	24    	[0.8]       	135	[1.]	[0.]	24    	[0.4]       	[3.92029347]	135	[4.15554762]	[3.77754903]	24    	[0.09568901]
136	21    	[0.9]       	136	[1.]	[0.]	21    	[0.3]       	[3.89034813]	136	[4.12985325]	[3.77754903]	21    	[0.115413]  
137	22    	[0.83333333]	137	[1.]	[0.]	22    	[0.372678]  	[3.89001558]	137	[4.13564253]	[3.77754903]	22    	[0.10355708]
138	20    	[0.76666667]	138	[1.]	[0.]	20    	[0.42295258]	[3.91056194]	138	[4.35990095]	[3.77754903]	20    	[0.14335983]
139	21    	[0.8]       	139	[1.]	[0.]	21    	[0.4]       	[3.90607706]	139	[4.23854542]	[3.77754903]	21    	[0.12484782]
140	18    	[0.66666667]	140	[1.]	[0.]	18    	[0.47140452]	[3.89955684]	140	[4.33197355]	[3.77754903]	18    	[0.11615417]
141	22    	[0.6]       	141	[1.]

201	17    	[0.86666667]	201	[1.]	[0.]	17    	[0.33993463]	[3.52263112]	201	[3.72115254]	[3.44130254]	17    	[0.0822864] 
202	20    	[0.9]       	202	[1.]	[0.]	20    	[0.3]       	[3.54503889]	202	[3.81706667]	[3.44130254]	20    	[0.11509169]
203	19    	[0.86666667]	203	[1.]	[0.]	19    	[0.33993463]	[3.55015761]	203	[3.79972839]	[3.43326306]	19    	[0.09501224]
204	16    	[0.93333333]	204	[1.]	[0.]	16    	[0.24944383]	[3.55838776]	204	[3.90484667]	[3.43326306]	16    	[0.13713029]
205	18    	[0.76666667]	205	[1.]	[0.]	18    	[0.42295258]	[3.54651073]	205	[3.95190859]	[3.43326306]	18    	[0.12717826]
206	19    	[0.86666667]	206	[1.]	[0.]	19    	[0.33993463]	[3.54232003]	206	[3.81608582]	[3.42875147]	19    	[0.12581265]
207	18    	[0.83333333]	207	[1.]	[0.]	18    	[0.372678]  	[3.53269178]	207	[3.81486011]	[3.42717218]	18    	[0.12453323]
208	14    	[0.9]       	208	[1.]	[0.]	14    	[0.3]       	[3.50241896]	208	[3.89544749]	[3.40185928]	14    	[0.11617757]
209	19    	[0.76666667]	209	[1.]

269	17    	[0.96666667]	269	[1.]	[0.]	17    	[0.17950549]	[3.46011839]	269	[3.78614593]	[3.35863042]	17    	[0.11172767]
270	19    	[0.9]       	270	[1.]	[0.]	19    	[0.3]       	[3.46457605]	270	[3.91949534]	[3.35863042]	19    	[0.12895478]
271	14    	[0.96666667]	271	[1.]	[0.]	14    	[0.17950549]	[3.43289544]	271	[3.60095978]	[3.35863042]	14    	[0.07637668]
272	17    	[0.93333333]	272	[1.]	[0.]	17    	[0.24944383]	[3.42189109]	272	[3.61235499]	[3.34304309]	17    	[0.07810367]
273	17    	[0.86666667]	273	[1.]	[0.]	17    	[0.33993463]	[3.44361374]	273	[3.71683812]	[3.34304309]	17    	[0.09453313]
274	21    	[0.9]       	274	[1.]	[0.]	21    	[0.3]       	[3.45868224]	274	[3.75696635]	[3.34304309]	21    	[0.10050268]
275	19    	[0.93333333]	275	[1.]	[0.]	19    	[0.24944383]	[3.47203391]	275	[3.77889943]	[3.34304309]	19    	[0.10665894]
276	20    	[0.86666667]	276	[1.]	[0.]	20    	[0.33993463]	[3.46397989]	276	[3.81094146]	[3.34304309]	20    	[0.12190502]
277	16    	[0.93333333]	277	[1.]

...

677	16    	[1.]        	677	[1.]	[1.]	16    	[0.]        	[3.252118]  	677	[3.67870784]	[3.12527013]	16    	[0.14683674]
678	18    	[1.]        	678	[1.]	[1.]	18    	[0.]        	[3.22573218]	678	[3.80291295]	[3.12043548]	18    	[0.13504934]
679	16    	[1.]        	679	[1.]	[1.]	16    	[0.]        	[3.22532129]	679	[3.47222972]	[3.12527013]	16    	[0.09946866]
680	18    	[0.96666667]	680	[1.]	[0.]	18    	[0.17950549]	[3.24059793]	680	[3.59493518]	[3.11983538]	18    	[0.12279087]
681	20    	[0.96666667]	681	[1.]	[0.]	20    	[0.17950549]	[3.26086946]	681	[3.7919395] 	[3.11983538]	20    	[0.148502]  
682	12    	[1.]        	682	[1.]	[1.]	12    	[0.]        	[3.19261007]	682	[3.39731288]	[3.11983538]	12    	[0.0842371] 
683	18    	[0.93333333]	683	[1.]	[0.]	18    	[0.24944383]	[3.20733248]	683	[3.53010249]	[3.10230541]	18    	[0.11319807]
684	19    	[1.]        	684	[1.]	[1.]	19    	[0.]        	[3.23528457]	684	[3.49855661]	[3.10901809]	19    	[0.09676824]
685	15    	[0.96666667]	685	[1.]

745	15    	[0.93333333]	745	[1.]	[0.]	15    	[0.24944383]	[3.21692855]	745	[3.47136664]	[3.10901809]	15    	[0.10804826]
746	23    	[1.]        	746	[1.]	[1.]	23    	[0.]        	[3.24172415]	746	[3.64199686]	[3.10901809]	23    	[0.13718222]
747	19    	[1.]        	747	[1.]	[1.]	19    	[0.]        	[3.20212259]	747	[3.49968195]	[3.10901809]	19    	[0.10064911]
748	16    	[1.]        	748	[1.]	[1.]	16    	[0.]        	[3.21535787]	748	[3.67252421]	[3.10901809]	16    	[0.1223691] 
749	17    	[1.]        	749	[1.]	[1.]	17    	[0.]        	[3.21257346]	749	[3.7166276] 	[3.10901809]	17    	[0.14518004]
750	19    	[1.]        	750	[1.]	[1.]	19    	[0.]        	[3.2143472] 	750	[3.65882564]	[3.10901809]	19    	[0.1340989] 
751	20    	[1.]        	751	[1.]	[1.]	20    	[0.]        	[3.20849374]	751	[3.59972382]	[3.10901809]	20    	[0.13115099]
752	16    	[1.]        	752	[1.]	[1.]	16    	[0.]        	[3.18310613]	752	[3.42666936]	[3.10279036]	16    	[0.08878043]
753	21    	[0.96666667]	753	[1.]

813	19    	[1.]        	813	[1.]	[1.]	19    	[0.]        	[3.1911369] 	813	[3.39778709]	[3.10279036]	19    	[0.09767451]
814	17    	[1.]        	814	[1.]	[1.]	17    	[0.]        	[3.22305548]	814	[3.56098104]	[3.10279036]	17    	[0.12048949]
815	16    	[1.]        	815	[1.]	[1.]	16    	[0.]        	[3.21928896]	815	[3.59125066]	[3.10279036]	16    	[0.14115727]
816	17    	[1.]        	816	[1.]	[1.]	17    	[0.]        	[3.22981919]	816	[3.4542129] 	[3.10279036]	17    	[0.11688745]
817	19    	[1.]        	817	[1.]	[1.]	19    	[0.]        	[3.25156429]	817	[3.70012593]	[3.10279036]	19    	[0.16312881]
818	24    	[1.]        	818	[1.]	[1.]	24    	[0.]        	[3.23273339]	818	[3.43490267]	[3.10279036]	24    	[0.09549112]
819	19    	[0.96666667]	819	[1.]	[0.]	19    	[0.17950549]	[3.25550745]	819	[3.66902733]	[3.10279036]	19    	[0.14007123]
820	15    	[1.]        	820	[1.]	[1.]	15    	[0.]        	[3.2219557] 	820	[3.59355235]	[3.10279036]	15    	[0.11994866]
821	21    	[0.96666667]	821	[1.]

881	18    	[1.]        	881	[1.]	[1.]	18    	[0.]        	[3.17518358]	881	[3.42467046]	[3.07009745]	18    	[0.10781973]
882	20    	[1.]        	882	[1.]	[1.]	20    	[0.]        	[3.15857728]	882	[3.40590835]	[3.07009745]	20    	[0.09821632]
883	16    	[1.]        	883	[1.]	[1.]	16    	[0.]        	[3.18297234]	883	[3.78837276]	[3.07009745]	16    	[0.15477733]
884	18    	[1.]        	884	[1.]	[1.]	18    	[0.]        	[3.1733597] 	884	[3.54385996]	[3.07009745]	18    	[0.11949744]
885	20    	[1.]        	885	[1.]	[1.]	20    	[0.]        	[3.13419208]	885	[3.42391133]	[3.07009745]	20    	[0.08819453]
886	20    	[0.96666667]	886	[1.]	[0.]	20    	[0.17950549]	[3.18928591]	886	[3.41446471]	[3.07009745]	20    	[0.11773424]
887	19    	[0.96666667]	887	[1.]	[0.]	19    	[0.17950549]	[3.17715805]	887	[3.67324138]	[3.06249213]	19    	[0.1370396] 
888	14    	[1.]        	888	[1.]	[1.]	14    	[0.]        	[3.1570836] 	888	[3.59369063]	[3.06249213]	14    	[0.13064445]
889	20    	[1.]        	889	[1.]

949	16    	[1.]        	949	[1.]	[1.]	16    	[0.]        	[3.17207354]	949	[3.44328117]	[3.04248881]	16    	[0.11916265]
950	13    	[1.]        	950	[1.]	[1.]	13    	[0.]        	[3.15000557]	950	[3.49077535]	[3.04248881]	13    	[0.12510472]
951	16    	[1.]        	951	[1.]	[1.]	16    	[0.]        	[3.13225158]	951	[3.45835686]	[3.04248881]	16    	[0.11186467]
952	19    	[1.]        	952	[1.]	[1.]	19    	[0.]        	[3.18071667]	952	[3.63587952]	[3.04248881]	19    	[0.14867359]
953	22    	[1.]        	953	[1.]	[1.]	22    	[0.]        	[3.16793074]	953	[3.52842712]	[3.04248881]	22    	[0.12030084]
954	18    	[1.]        	954	[1.]	[1.]	18    	[0.]        	[3.15952663]	954	[3.45285034]	[3.04248881]	18    	[0.11648284]
955	19    	[1.]        	955	[1.]	[1.]	19    	[0.]        	[3.18942169]	955	[3.43338823]	[3.04248881]	19    	[0.13087505]
956	20    	[1.]        	956	[1.]	[1.]	20    	[0.]        	[3.20863484]	956	[3.43559456]	[3.04248881]	20    	[0.13666225]
957	25    	[1.]        	957	[1.]

In [235]:
data_idx = 615
original_points, target_fv, target = GA_dataset.__getitem__(data_idx)
original_points = torch.from_numpy(original_points.astype(np.float32)).clone().view(1, 1024, 3)
target_fv = torch.from_numpy(target_fv.astype(np.float32)).clone().view(1, -1)
target = torch.from_numpy(np.array(target).astype(np.float32)).clone()

Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cuda()
Feature_vector_calculator = Feature_vector_calculator.eval().cuda()

rate = 1/16
ngen = 1000
seed = 0
cxpd = 0.2
mutpd = 0.5

hof, pop, log= run_GA(original_points, target_fv, target, Feature_vector_calculator, \
                      Classifier_from_feature_vector, rate, ngen, seed, cxpd, mutpd)
save_GA_results(log, hof, pop, 'seed0', 'GA_downsampling',data=data_idx, ngen=ngen, rate=rate, seed=seed)

   	      	                            acc                             	                                    dist                                   
   	      	------------------------------------------------------------	---------------------------------------------------------------------------
gen	nevals	avg         	gen	max 	min 	nevals	std         	avg         	gen	max         	min         	nevals	std        
0  	30    	[0.13333333]	0  	[1.]	[0.]	30    	[0.33993463]	[8.73670786]	0  	[9.59622097]	[7.68606949]	30    	[0.4441062]
1  	19    	[0.13333333]	1  	[1.]	[0.]	19    	[0.33993463]	[8.45532316]	1  	[9.01413631]	[7.68606949]	19    	[0.32119019]
2  	21    	[0.1]       	2  	[1.]	[0.]	21    	[0.3]       	[8.21049239]	2  	[8.89145851]	[7.50666904]	21    	[0.33326169]
3  	21    	[0.06666667]	3  	[1.]	[0.]	21    	[0.24944383]	[8.00663384]	3  	[9.13956165]	[7.49649382]	21    	[0.39037248]
4  	15    	[0.06666667]	4  	[1.]	[0.]	15    	[0.24944383]	[7.74766649]	4  	[8.33772182]	[7.46969748]	

65 	17    	[0.]        	65 	[0.]	[0.]	17    	[0.]        	[5.66562055]	65 	[6.07291794]	[5.44391632]	17    	[0.17529726]
66 	16    	[0.]        	66 	[0.]	[0.]	16    	[0.]        	[5.62676781]	66 	[6.20279837]	[5.44391632]	16    	[0.18180713]
67 	20    	[0.]        	67 	[0.]	[0.]	20    	[0.]        	[5.62745668]	67 	[6.10305166]	[5.44391632]	20    	[0.18197301]
68 	19    	[0.03333333]	68 	[1.]	[0.]	19    	[0.17950549]	[5.62738463]	68 	[6.06697035]	[5.40656185]	19    	[0.1831226] 
69 	17    	[0.]        	69 	[0.]	[0.]	17    	[0.]        	[5.60105921]	69 	[6.01893187]	[5.30942106]	17    	[0.18713661]
70 	18    	[0.]        	70 	[0.]	[0.]	18    	[0.]        	[5.62425086]	70 	[6.12877846]	[5.30942106]	18    	[0.21320657]
71 	19    	[0.03333333]	71 	[1.]	[0.]	19    	[0.17950549]	[5.62840233]	71 	[6.22644281]	[5.36385107]	19    	[0.22555259]
72 	16    	[0.13333333]	72 	[1.]	[0.]	16    	[0.33993463]	[5.60681753]	72 	[6.2384758] 	[5.36385107]	16    	[0.22047245]
73 	19    	[0.06666667]	73 	[1.]

133	17    	[0.23333333]	133	[1.]	[0.]	17    	[0.42295258]	[4.96224567]	133	[5.38090658]	[4.79175329]	17    	[0.16109948]
134	14    	[0.16666667]	134	[1.]	[0.]	14    	[0.372678]  	[4.9292528] 	134	[5.49414015]	[4.79175329]	14    	[0.17430573]
135	25    	[0.16666667]	135	[1.]	[0.]	25    	[0.372678]  	[5.02564789]	135	[5.49529123]	[4.79175329]	25    	[0.23519806]
136	17    	[0.2]       	136	[1.]	[0.]	17    	[0.4]       	[4.94851812]	136	[5.39778662]	[4.79175329]	17    	[0.19178574]
137	18    	[0.26666667]	137	[1.]	[0.]	18    	[0.44221664]	[5.00480418]	137	[5.41076231]	[4.79175329]	18    	[0.19010973]
138	16    	[0.23333333]	138	[1.]	[0.]	16    	[0.42295258]	[4.96669057]	138	[5.43758774]	[4.79175329]	16    	[0.17751995]
139	18    	[0.13333333]	139	[1.]	[0.]	18    	[0.33993463]	[4.96147122]	139	[5.55895901]	[4.77215767]	18    	[0.20021082]
140	18    	[0.06666667]	140	[1.]	[0.]	18    	[0.24944383]	[4.93877339]	140	[5.34372902]	[4.79175329]	18    	[0.14482619]
141	18    	[0.2]       	141	[1.]

201	16    	[0.76666667]	201	[1.]	[0.]	16    	[0.42295258]	[4.81092696]	201	[5.2371273] 	[4.66410685]	16    	[0.17588971]
202	20    	[0.63333333]	202	[1.]	[0.]	20    	[0.48189441]	[4.78793268]	202	[5.17001867]	[4.66410685]	20    	[0.14412899]
203	12    	[0.76666667]	203	[1.]	[0.]	12    	[0.42295258]	[4.83315337]	203	[5.32427454]	[4.66410685]	12    	[0.22757077]
204	25    	[0.63333333]	204	[1.]	[0.]	25    	[0.48189441]	[4.92632073]	204	[5.78168011]	[4.66410685]	25    	[0.26655173]
205	23    	[0.66666667]	205	[1.]	[0.]	23    	[0.47140452]	[4.87209617]	205	[5.38772011]	[4.66410685]	23    	[0.18846317]
206	22    	[0.76666667]	206	[1.]	[0.]	22    	[0.42295258]	[4.82605453]	206	[5.25877714]	[4.66410685]	22    	[0.14411275]
207	19    	[0.63333333]	207	[1.]	[0.]	19    	[0.48189441]	[4.83258613]	207	[5.18853235]	[4.66410685]	19    	[0.15604172]
208	18    	[0.66666667]	208	[1.]	[0.]	18    	[0.47140452]	[4.81411246]	208	[5.15719986]	[4.66410685]	18    	[0.13515109]
209	14    	[0.6]       	209	[1.]

269	20    	[0.8]       	269	[1.]	[0.]	20    	[0.4]       	[4.85044454]	269	[5.43060064]	[4.66026163]	20    	[0.19411624]
270	18    	[0.7]       	270	[1.]	[0.]	18    	[0.45825757]	[4.87733749]	270	[5.38122463]	[4.66026163]	18    	[0.23236648]
271	24    	[0.66666667]	271	[1.]	[0.]	24    	[0.47140452]	[4.90896339]	271	[5.41554642]	[4.66026163]	24    	[0.22175041]
272	21    	[0.6]       	272	[1.]	[0.]	21    	[0.48989795]	[4.94132972]	272	[5.41537857]	[4.66026163]	21    	[0.25471058]
273	19    	[0.63333333]	273	[1.]	[0.]	19    	[0.48189441]	[4.9560125] 	273	[5.76173735]	[4.66026163]	19    	[0.28126246]
274	11    	[0.86666667]	274	[1.]	[0.]	11    	[0.33993463]	[4.88559615]	274	[5.44524288]	[4.66026163]	11    	[0.21957476]
275	17    	[0.86666667]	275	[1.]	[0.]	17    	[0.33993463]	[4.86006327]	275	[5.52620649]	[4.66026163]	17    	[0.237751]  
276	21    	[0.66666667]	276	[1.]	[0.]	21    	[0.47140452]	[4.90356696]	276	[5.32647133]	[4.66026163]	21    	[0.2148483] 
277	16    	[0.7]       	277	[1.]

...

677	17    	[0.16666667]	677	[1.]	[0.]	17    	[0.372678]  	[4.68758101]	677	[5.08101511]	[4.5723505] 	17    	[0.14915519]
678	21    	[0.2]       	678	[1.]	[0.]	21    	[0.4]       	[4.71016984]	678	[5.15698242]	[4.5723505] 	21    	[0.17667636]
679	18    	[0.16666667]	679	[1.]	[0.]	18    	[0.372678]  	[4.68410266]	679	[5.04631567]	[4.5723505] 	18    	[0.1380778] 
680	20    	[0.33333333]	680	[1.]	[0.]	20    	[0.47140452]	[4.72620866]	680	[5.10044432]	[4.5723505] 	20    	[0.17229371]
681	20    	[0.5]       	681	[1.]	[0.]	20    	[0.5]       	[4.74236986]	681	[5.17604399]	[4.5723505] 	20    	[0.1880053] 
682	18    	[0.33333333]	682	[1.]	[0.]	18    	[0.47140452]	[4.76718683]	682	[5.20424843]	[4.5723505] 	18    	[0.19842002]
683	16    	[0.3]       	683	[1.]	[0.]	16    	[0.45825757]	[4.71852744]	683	[5.1798768] 	[4.5723505] 	16    	[0.17039801]
684	21    	[0.3]       	684	[1.]	[0.]	21    	[0.45825757]	[4.74038582]	684	[5.18424606]	[4.5723505] 	21    	[0.18061376]
685	16    	[0.13333333]	685	[1.]

745	16    	[0.73333333]	745	[1.]	[0.]	16    	[0.44221664]	[4.77413411]	745	[5.25690556]	[4.57114887]	16    	[0.21164266]
746	20    	[0.83333333]	746	[1.]	[0.]	20    	[0.372678]  	[4.78591633]	746	[5.32292509]	[4.57114887]	20    	[0.2478709] 
747	16    	[0.76666667]	747	[1.]	[0.]	16    	[0.42295258]	[4.78509324]	747	[5.54714775]	[4.57114887]	16    	[0.22791721]
748	21    	[0.83333333]	748	[1.]	[0.]	21    	[0.372678]  	[4.78346388]	748	[5.18364191]	[4.57114887]	21    	[0.18847907]
749	14    	[0.7]       	749	[1.]	[0.]	14    	[0.45825757]	[4.79303886]	749	[5.38444376]	[4.57114887]	14    	[0.24463319]
750	20    	[0.8]       	750	[1.]	[0.]	20    	[0.4]       	[4.82655376]	750	[5.3150444] 	[4.57114887]	20    	[0.2087632] 
751	20    	[0.8]       	751	[1.]	[0.]	20    	[0.4]       	[4.85703479]	751	[5.54592419]	[4.57114887]	20    	[0.24195036]
752	16    	[0.76666667]	752	[1.]	[0.]	16    	[0.42295258]	[4.83716351]	752	[5.31058264]	[4.57114887]	16    	[0.19296571]
753	17    	[0.63333333]	753	[1.]

813	16    	[0.8]       	813	[1.]	[0.]	16    	[0.4]       	[4.75253085]	813	[5.24564981]	[4.57114887]	16    	[0.17547621]
814	18    	[0.76666667]	814	[1.]	[0.]	18    	[0.42295258]	[4.73206868]	814	[5.2730279] 	[4.5605402] 	18    	[0.18213831]
815	16    	[0.8]       	815	[1.]	[0.]	16    	[0.4]       	[4.74435201]	815	[5.1528964] 	[4.5605402] 	16    	[0.17975551]
816	24    	[0.83333333]	816	[1.]	[0.]	24    	[0.372678]  	[4.75580896]	816	[5.12617588]	[4.5605402] 	24    	[0.16586829]
817	15    	[0.9]       	817	[1.]	[0.]	15    	[0.3]       	[4.75689314]	817	[5.07704926]	[4.5605402] 	15    	[0.17886889]
818	16    	[0.93333333]	818	[1.]	[0.]	16    	[0.24944383]	[4.74278529]	818	[5.28039885]	[4.5605402] 	16    	[0.21158496]
819	18    	[0.9]       	819	[1.]	[0.]	18    	[0.3]       	[4.74407323]	819	[5.36733484]	[4.5605402] 	18    	[0.19196701]
820	19    	[0.8]       	820	[1.]	[0.]	19    	[0.4]       	[4.73027205]	820	[4.99267769]	[4.5605402] 	19    	[0.13143266]
821	15    	[0.86666667]	821	[1.]

881	21    	[0.86666667]	881	[1.]	[0.]	21    	[0.33993463]	[4.7608542] 	881	[5.4301281] 	[4.55273962]	21    	[0.21895728]
882	19    	[0.9]       	882	[1.]	[0.]	19    	[0.3]       	[4.76592175]	882	[5.32823181]	[4.55273962]	19    	[0.2084038] 
883	14    	[0.9]       	883	[1.]	[0.]	14    	[0.3]       	[4.77277946]	883	[5.34551048]	[4.55273962]	14    	[0.23507525]
884	23    	[0.86666667]	884	[1.]	[0.]	23    	[0.33993463]	[4.78293308]	884	[5.55184841]	[4.55273962]	23    	[0.28816521]
885	18    	[0.9]       	885	[1.]	[0.]	18    	[0.3]       	[4.75651482]	885	[5.2192049] 	[4.55273962]	18    	[0.19464671]
886	19    	[0.93333333]	886	[1.]	[0.]	19    	[0.24944383]	[4.69439324]	886	[5.18604279]	[4.55273962]	19    	[0.17791426]
887	20    	[0.96666667]	887	[1.]	[0.]	20    	[0.17950549]	[4.71947643]	887	[5.33286858]	[4.55273962]	20    	[0.20319675]
888	17    	[0.93333333]	888	[1.]	[0.]	17    	[0.24944383]	[4.69810985]	888	[5.05856514]	[4.55273962]	17    	[0.15759726]
889	20    	[0.93333333]	889	[1.]

949	20    	[0.93333333]	949	[1.]	[0.]	20    	[0.24944383]	[4.7504334] 	949	[5.28726673]	[4.53806639]	20    	[0.22191604]
950	12    	[0.96666667]	950	[1.]	[0.]	12    	[0.17950549]	[4.6801153] 	950	[5.2014575] 	[4.53806639]	12    	[0.21120313]
951	18    	[0.96666667]	951	[1.]	[0.]	18    	[0.17950549]	[4.67454548]	951	[5.11588383]	[4.53806639]	18    	[0.17256239]
952	20    	[0.9]       	952	[1.]	[0.]	20    	[0.3]       	[4.67424614]	952	[5.16827488]	[4.53806639]	20    	[0.18327251]
953	16    	[0.96666667]	953	[1.]	[0.]	16    	[0.17950549]	[4.74936616]	953	[5.32833672]	[4.53806639]	16    	[0.25471115]
954	17    	[0.9]       	954	[1.]	[0.]	17    	[0.3]       	[4.68352869]	954	[5.33843803]	[4.53806639]	17    	[0.20498848]
955	18    	[0.96666667]	955	[1.]	[0.]	18    	[0.17950549]	[4.73040964]	955	[5.12229252]	[4.53806639]	18    	[0.18983796]
956	19    	[0.96666667]	956	[1.]	[0.]	19    	[0.17950549]	[4.77832535]	956	[5.22351789]	[4.53806639]	19    	[0.21212834]
957	22    	[0.86666667]	957	[1.]

In [236]:
data_idx = 615
original_points, target_fv, target = GA_dataset.__getitem__(data_idx)
original_points = torch.from_numpy(original_points.astype(np.float32)).clone().view(1, 1024, 3)
target_fv = torch.from_numpy(target_fv.astype(np.float32)).clone().view(1, -1)
target = torch.from_numpy(np.array(target).astype(np.float32)).clone()

Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cuda()
Feature_vector_calculator = Feature_vector_calculator.eval().cuda()

rate = 1/32
ngen = 1000
seed = 0
cxpd = 0.2
mutpd = 0.5

hof, pop, log= run_GA(original_points, target_fv, target, Feature_vector_calculator, \
                      Classifier_from_feature_vector, rate, ngen, seed, cxpd, mutpd)
save_GA_results(log, hof, pop, 'seed0', 'GA_downsampling',data=data_idx, ngen=ngen, rate=rate, seed=seed)

   	      	                     acc                     	                                    dist                                    
   	      	---------------------------------------------	----------------------------------------------------------------------------
gen	nevals	avg  	gen	max 	min 	nevals	std  	avg          	gen	max          	min          	nevals	std         
0  	30    	[0.1]	0  	[1.]	[0.]	30    	[0.3]	[11.61218433]	0  	[12.89272308]	[10.66214275]	30    	[0.44360748]
1  	20    	[0.03333333]	1  	[1.]	[0.]	20    	[0.17950549]	[11.30197884]	1  	[12.97142696]	[10.48063374]	20    	[0.4545588] 
2  	18    	[0.]        	2  	[0.]	[0.]	18    	[0.]        	[11.09211289]	2  	[11.58520794]	[10.48063374]	18    	[0.29185201]
3  	9     	[0.]        	3  	[0.]	[0.]	9     	[0.]        	[10.92819001]	3  	[11.22920704]	[10.48063374]	9     	[0.20309982]
4  	21    	[0.]        	4  	[0.]	[0.]	21    	[0.]        	[10.88961493]	4  	[11.41094017]	[10.4560051] 	21    	[0.25472658]
5  	16    	[0.] 

64 	20    	[0.16666667]	64 	[1.]	[0.]	20    	[0.372678]  	[9.07494342] 	64 	[9.42562389] 	[8.91112709] 	20    	[0.14981594]
65 	19    	[0.26666667]	65 	[1.]	[0.]	19    	[0.44221664]	[9.1089633]  	65 	[9.79265213] 	[8.86204433] 	19    	[0.21214035]
66 	21    	[0.33333333]	66 	[1.]	[0.]	21    	[0.47140452]	[9.08414539] 	66 	[9.6079092]  	[8.86204433] 	21    	[0.17083584]
67 	18    	[0.26666667]	67 	[1.]	[0.]	18    	[0.44221664]	[9.05388187] 	67 	[9.36977291] 	[8.84730434] 	18    	[0.15369975]
68 	15    	[0.36666667]	68 	[1.]	[0.]	15    	[0.48189441]	[9.03722207] 	68 	[9.70253277] 	[8.7876091]  	15    	[0.23608717]
69 	16    	[0.43333333]	69 	[1.]	[0.]	16    	[0.49553562]	[9.0288524]  	69 	[9.98524666] 	[8.7876091]  	16    	[0.25375302]
70 	16    	[0.33333333]	70 	[1.]	[0.]	16    	[0.47140452]	[8.98483747] 	70 	[9.56463814] 	[8.7876091]  	16    	[0.17240418]
71 	16    	[0.5]       	71 	[1.]	[0.]	16    	[0.5]       	[8.99360758] 	71 	[9.82249355] 	[8.77494335] 	16    	[0.240694]  
72 	17  

131	17    	[0.13333333]	131	[1.]	[0.]	17    	[0.33993463]	[8.63394613] 	131	[9.35039711] 	[8.39904213] 	17    	[0.2362552] 
132	19    	[0.2]       	132	[1.]	[0.]	19    	[0.4]       	[8.65443786] 	132	[9.17834949] 	[8.39904213] 	19    	[0.21870695]
133	16    	[0.1]       	133	[1.]	[0.]	16    	[0.3]       	[8.61409566] 	133	[9.11885262] 	[8.39904213] 	16    	[0.2193684] 
134	17    	[0.16666667]	134	[1.]	[0.]	17    	[0.372678]  	[8.58742453] 	134	[9.83768177] 	[8.39904213] 	17    	[0.32217513]
135	17    	[0.03333333]	135	[1.]	[0.]	17    	[0.17950549]	[8.56355244] 	135	[9.30508137] 	[8.39904213] 	17    	[0.2382624] 
136	18    	[0.03333333]	136	[1.]	[0.]	18    	[0.17950549]	[8.55538766] 	136	[8.95663452] 	[8.38336468] 	18    	[0.18278797]
137	20    	[0.13333333]	137	[1.]	[0.]	20    	[0.33993463]	[8.6654631]  	137	[9.61079597] 	[8.38336468] 	20    	[0.34119169]
138	19    	[0.23333333]	138	[1.]	[0.]	19    	[0.42295258]	[8.60431563] 	138	[9.02599239] 	[8.38336468] 	19    	[0.18744493]
139	16  

198	18    	[0.06666667]	198	[1.]	[0.]	18    	[0.24944383]	[8.5523318]  	198	[9.21158028] 	[8.34714794] 	18    	[0.23708007]
199	16    	[0.06666667]	199	[1.]	[0.]	16    	[0.24944383]	[8.53645506] 	199	[9.41027832] 	[8.34714794] 	16    	[0.27279237]
200	18    	[0.1]       	200	[1.]	[0.]	18    	[0.3]       	[8.58891255] 	200	[9.8397131]  	[8.34714794] 	18    	[0.32618502]
201	15    	[0.]        	201	[0.]	[0.]	15    	[0.]        	[8.45216061] 	201	[9.11340237] 	[8.34714794] 	15    	[0.17377176]
202	13    	[0.13333333]	202	[1.]	[0.]	13    	[0.33993463]	[8.47019545] 	202	[9.06840992] 	[8.34714794] 	13    	[0.2071642] 
203	17    	[0.06666667]	203	[1.]	[0.]	17    	[0.24944383]	[8.46417637] 	203	[8.95492458] 	[8.34714794] 	17    	[0.18331767]
204	17    	[0.1]       	204	[1.]	[0.]	17    	[0.3]       	[8.50187654] 	204	[9.10615826] 	[8.28116894] 	17    	[0.20658664]
205	17    	[0.03333333]	205	[1.]	[0.]	17    	[0.17950549]	[8.55627673] 	205	[10.18619823]	[8.28116894] 	17    	[0.37400855]
206	19  

265	20    	[0.2]       	265	[1.]	[0.]	20    	[0.4]       	[8.23803746] 	265	[9.69283772] 	[7.97706985] 	20    	[0.33176501]
266	19    	[0.16666667]	266	[1.]	[0.]	19    	[0.372678]  	[8.21381456] 	266	[8.83798122] 	[7.97706985] 	19    	[0.1994605] 
267	21    	[0.1]       	267	[1.]	[0.]	21    	[0.3]       	[8.29314558] 	267	[9.61555099] 	[7.97706985] 	21    	[0.30080078]
268	15    	[0.1]       	268	[1.]	[0.]	15    	[0.3]       	[8.2484196]  	268	[9.49527359] 	[7.97706985] 	15    	[0.31673006]
269	17    	[0.13333333]	269	[1.]	[0.]	17    	[0.33993463]	[8.28044283] 	269	[9.49847603] 	[7.97706985] 	17    	[0.34098726]
270	20    	[0.23333333]	270	[1.]	[0.]	20    	[0.42295258]	[8.29146434] 	270	[9.49668503] 	[7.97706985] 	20    	[0.33679877]
271	16    	[0.13333333]	271	[1.]	[0.]	16    	[0.33993463]	[8.22300704] 	271	[8.85968685] 	[7.97706985] 	16    	[0.25380603]
272	17    	[0.23333333]	272	[1.]	[0.]	17    	[0.42295258]	[8.21865708] 	272	[8.95412159] 	[7.97417974] 	17    	[0.27731416]
273	22  

...

734	22    	[0.36666667]	734	[1.]	[0.]	22    	[0.48189441]	[7.90787489] 	734	[8.39375305] 	[7.60646009] 	22    	[0.27408009]
735	14    	[0.26666667]	735	[1.]	[0.]	14    	[0.44221664]	[7.85791149] 	735	[8.68309689] 	[7.60646009] 	14    	[0.27082342]
736	18    	[0.23333333]	736	[1.]	[0.]	18    	[0.42295258]	[7.85997585] 	736	[8.37080669] 	[7.60646009] 	18    	[0.25448525]
737	16    	[0.13333333]	737	[1.]	[0.]	16    	[0.33993463]	[7.83878209] 	737	[9.31213284] 	[7.60646009] 	16    	[0.33496679]
738	15    	[0.03333333]	738	[1.]	[0.]	15    	[0.17950549]	[7.76596945] 	738	[8.55408669] 	[7.60646009] 	15    	[0.22545515]
739	20    	[0.26666667]	739	[1.]	[0.]	20    	[0.44221664]	[7.80787533] 	739	[8.57497025] 	[7.60646009] 	20    	[0.25924923]
740	17    	[0.16666667]	740	[1.]	[0.]	17    	[0.372678]  	[7.75620936] 	740	[8.02826405] 	[7.60334873] 	17    	[0.15318418]
741	13    	[0.2]       	741	[1.]	[0.]	13    	[0.4]       	[7.79647183] 	741	[8.40850353] 	[7.60334873] 	13    	[0.23711909]
742	22  

801	18    	[0.06666667]	801	[1.]	[0.]	18    	[0.24944383]	[7.89230955] 	801	[8.60172749] 	[7.60063648] 	18    	[0.30569766]
802	19    	[0.03333333]	802	[1.]	[0.]	19    	[0.17950549]	[7.83225234] 	802	[8.57611275] 	[7.60063648] 	19    	[0.22606432]
803	14    	[0.03333333]	803	[1.]	[0.]	14    	[0.17950549]	[7.83151793] 	803	[8.53422451] 	[7.59320593] 	14    	[0.30753898]
804	19    	[0.]        	804	[0.]	[0.]	19    	[0.]        	[7.88603217] 	804	[9.3840704]  	[7.60063648] 	19    	[0.39934721]
805	15    	[0.]        	805	[0.]	[0.]	15    	[0.]        	[7.82363113] 	805	[8.20845985] 	[7.60063648] 	15    	[0.19681202]
806	20    	[0.]        	806	[0.]	[0.]	20    	[0.]        	[7.87575782] 	806	[8.38850784] 	[7.60063648] 	20    	[0.26539898]
807	18    	[0.]        	807	[0.]	[0.]	18    	[0.]        	[7.75718681] 	807	[8.38776779] 	[7.60063648] 	18    	[0.21485535]
808	20    	[0.]        	808	[0.]	[0.]	20    	[0.]        	[7.89727117] 	808	[9.49956512] 	[7.60063648] 	20    	[0.39884296]
809	20  

868	20    	[0.16666667]	868	[1.]	[0.]	20    	[0.372678]  	[7.81410462] 	868	[8.32431984] 	[7.5619216]  	20    	[0.25326343]
869	12    	[0.13333333]	869	[1.]	[0.]	12    	[0.33993463]	[7.76964728] 	869	[8.41004848] 	[7.5619216]  	12    	[0.26161928]
870	13    	[0.03333333]	870	[1.]	[0.]	13    	[0.17950549]	[7.72955842] 	870	[8.69570827] 	[7.5619216]  	13    	[0.27105023]
871	16    	[0.1]       	871	[1.]	[0.]	16    	[0.3]       	[7.69666349] 	871	[8.28380966] 	[7.5619216]  	16    	[0.2151287] 
872	15    	[0.16666667]	872	[1.]	[0.]	15    	[0.372678]  	[7.79805705] 	872	[9.05085754] 	[7.5619216]  	15    	[0.36224312]
873	16    	[0.13333333]	873	[1.]	[0.]	16    	[0.33993463]	[7.73420563] 	873	[8.19182587] 	[7.54295969] 	16    	[0.23456561]
874	21    	[0.2]       	874	[1.]	[0.]	21    	[0.4]       	[7.84819333] 	874	[9.2091856]  	[7.54295969] 	21    	[0.36360348]
875	16    	[0.23333333]	875	[1.]	[0.]	16    	[0.42295258]	[7.80147999] 	875	[8.29109859] 	[7.54295969] 	16    	[0.2399866] 
876	21  

935	16    	[0.13333333]	935	[1.]	[0.]	16    	[0.33993463]	[7.66181779] 	935	[8.08413982] 	[7.50522089] 	16    	[0.19156758]
936	18    	[0.26666667]	936	[1.]	[0.]	18    	[0.44221664]	[7.74115074] 	936	[8.66594601] 	[7.50522089] 	18    	[0.27370166]
937	21    	[0.23333333]	937	[1.]	[0.]	21    	[0.42295258]	[7.69574375] 	937	[8.44808388] 	[7.50522089] 	21    	[0.24831631]
938	17    	[0.26666667]	938	[1.]	[0.]	17    	[0.44221664]	[7.73246299] 	938	[8.497715]   	[7.50522089] 	17    	[0.29180343]
939	16    	[0.2]       	939	[1.]	[0.]	16    	[0.4]       	[7.74854334] 	939	[8.38749695] 	[7.50522089] 	16    	[0.24283354]
940	22    	[0.43333333]	940	[1.]	[0.]	22    	[0.49553562]	[7.89762066] 	940	[8.5332737]  	[7.50522089] 	22    	[0.32568308]
941	21    	[0.13333333]	941	[1.]	[0.]	21    	[0.33993463]	[7.84271019] 	941	[8.73034096] 	[7.50522089] 	21    	[0.32003929]
942	8     	[0.16666667]	942	[1.]	[0.]	8     	[0.372678]  	[7.74324794] 	942	[8.18220901] 	[7.50522089] 	8     	[0.24599569]
943	18  

Saved in: /mnt/workspace/Github/Pointnet_Pointnet2_pytorch/Mywork/Results/Files/seed0/GA_downsampling/log/data615_ngen1000_rate0.03125_seed0_2022-01-04_20-40.csv
Saved in: /mnt/workspace/Github/Pointnet_Pointnet2_pytorch/Mywork/Results/Files/seed0/GA_downsampling/pop/data615_ngen1000_rate0.03125_seed0_2022-01-04_20-40.csv


# これ以降は付録

# 分類のランダム性のチェック

In [17]:
data_idx = 197
miss = np.empty(0)
dist_arr = np.empty(0)


points, target_fv, target = GA_dataset.__getitem__(data_idx)
points = torch.from_numpy(points.astype(np.float32)).clone().view(1, 1024, 3).transpose(1, 2)
# print(points.shape)
target_fv = torch.from_numpy(target_fv.astype(np.float32)).clone().view(1, -1)
# print("{}{: ^7}{}{: ^7}{}{: ^7}{}{: ^7}{}".format('|','Dist','|', 'Ans','|', 'Pred', '|', 'Correct', '|'))
# print("---------------------------------")

for i in range(100):
    initialize(i)
    with torch.no_grad():
        Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cpu()
        Feature_vector_calculator = Feature_vector_calculator.eval().cpu()

        feature_vector = Feature_vector_calculator(points)
        dist = torch.sum(torch.cdist(feature_vector.view(1, 1, -1), target_fv.view(1, 1, -1)), dim=0)
    #     score = torch.cat((score, dist[0]), 0)
    #     print("Distance of feature vector: ", dist[0])

        pred, _ = Classifier_from_feature_vector(feature_vector)
        pred_choice = pred.data.max(1)[1].detach().numpy().astype(np.int32)



#     print("{}{: ^.5f}{}{: ^7}{}{: ^7}{}{: ^7}{}".format('|', float(dist[0]),'|', str(target),'|', str(pred_choice[0]), '|', target == pred_choice[0], '|'))  
#     print("Dist: {0}  Answer: {1}   Pred: {1}   {2}".format(target, pred_choice[0], target == pred_choice))
    if target != pred_choice[0]:
        miss = np.append(miss, 0)
    else:
        miss = np.append(miss, 1)
    dist_arr = np.append(dist_arr, dist)
# if miss > 0:
#     print("{} : {}".format(data_idx, miss))

| Dist  |  Ans  | Pred  |Correct|
---------------------------------
|1.86695|   2   |   2   |   1   |
|1.69112|   2   |   2   |   1   |
|1.94310|   2   |   2   |   1   |
|2.04167|   2   |   2   |   1   |
|2.03489|   2   |   2   |   1   |
|1.93824|   2   |   2   |   1   |
|1.84395|   2   |   2   |   1   |
|1.83608|   2   |   2   |   1   |
|1.98636|   2   |   2   |   1   |
|1.89502|   2   |   2   |   1   |
|1.84507|   2   |   2   |   1   |
|1.91427|   2   |   2   |   1   |
|1.78525|   2   |   2   |   1   |
|1.80117|   2   |   2   |   1   |
|1.86896|   2   |   2   |   1   |
|1.72689|   2   |   2   |   1   |
|1.75888|   2   |   2   |   1   |
|1.91797|   2   |   2   |   1   |
|1.96264|   2   |   2   |   1   |
|1.87928|   2   |   2   |   1   |
|1.91871|   2   |   2   |   1   |
|1.90087|   2   |   2   |   1   |
|2.00837|   2   |   2   |   1   |
|1.88893|   2   |   2   |   1   |
|1.85505|   2   |   2   |   1   |
|1.92150|   2   |   2   |   1   |
|1.68540|   2   |   2   |   1   |
|2.08213|   2 

In [ ]:
rate = 0.5
ngen = 1000
seed = 0
hof, pop, log = run_GA(GADataLoader, Feature_vector_calculator, Classifier_from_feature_vector, rate, ngen=ngen, seed=seed, cxpd=0.5, mutpd=0.2)
save_GA_results(log, hof, ngen, rate, seed)

In [14]:
idx = create_select_idx(1024, 512)
print(len(idx))

seed = 0
initialize(seed)
score = fitness(idx, GADataLoader, Feature_vector_calculator)

512


# 正答率の高いテストデータのチェック

In [345]:
prediction_accuracy = np.empty(0)
iteration = 100

initialize(0)
for i, (points, _, target) in tqdm(enumerate(GADataLoader), total=len(GADataLoader)):
    B, _, _ = points.shape
    points, target = points.cuda(), target.cuda()
    points = points.transpose(2, 1)
#     points = torch.from_numpy(points.astype(np.float32)).clone().view(1, 1024, 3).transpose(1, 2)
    with torch.no_grad():
        Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cuda()
        Feature_vector_calculator = Feature_vector_calculator.eval().cuda()
        
        correct_arr = torch.empty(0).cpu()
        for _ in range(iteration):
            feature_vector = Feature_vector_calculator(points)
            pred, _ = Classifier_from_feature_vector(feature_vector)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.long().data).cpu()
            correct_arr = torch.cat((correct_arr, correct), dim=0)
    correct_arr = correct_arr.view(iteration, B)#(100, 103)
    correct_arr = correct_arr.detach().numpy().astype(np.int32)
    correct_arr = np.mean(correct_arr, axis=0) #バッチの正答率配列
    prediction_accuracy = np.append(prediction_accuracy, correct_arr)
    
# prediction_accuracy = prediction_accuracy.reshape(2468)
print(prediction_accuracy)

 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▎    | 23/24 [14:55<00:38, 38.94s/it]


RuntimeError: shape '[100, 103]' is invalid for input of size 9900

In [350]:
print(prediction_accuracy)
print(np.mean(prediction_accuracy))
pa_df = pd.DataFrame(data=prediction_accuracy)


[1. 1. 1. ... 1. 1. 1.]
0.9223987034035654


In [352]:
print(prediction_accuracy[855])

0.88


In [351]:
method_name = 'seed0/each_frame_accuracy' #何の結果か、どのような加工を加えているか、何個ずつか
prefix = ''
ext = ".csv"
df = pa_df
#-------------------------------------
save_dir = FILE_DIR / Path(method_name) 
# print(save_dir)
save_dir.mkdir(exist_ok=True, parents=True)
timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
# print(timestr)
file_name = prefix + timestr + ext
df.to_csv(save_dir/file_name, index=False)
print("Saved in:", save_dir/file_name)

Saved in: /mnt/workspace/Github/Pointnet_Pointnet2_pytorch/Mywork/Results/Files/seed0/each_frame_accuracy/2021-12-15_12-06.csv


# ベース特徴ベクトルの計算

# ベース特徴での各フレームの正誤

In [365]:
fv_path = FILE_DIR/'seed0'/'Feature_vector_base'/'2021-12-07_13-27.csv'
fv = pd.read_csv(fv_path)

In [366]:
print(fv.shape)
print(fv[:103].shape)
print(fv.values.dtype)

(2468, 1024)
(103, 1024)
float64


In [367]:
fv = torch.from_numpy(fv.values.astype(np.float32)).clone().cuda()

In [368]:
prediction_accuracy = np.empty(0)
iteration = 1
initialize(0)
first = 0
for i, (points, _, target) in tqdm(enumerate(GADataLoader), total=len(GADataLoader)):
    B, _, _ = points.shape
    target = target.cuda()
    last = first + B
#     points = points.transpose(2, 1)
#     points = torch.from_numpy(points.astype(np.float32)).clone().view(1, 1024, 3).transpose(1, 2)
    with torch.no_grad():
#         Feature_vector_calculator = Feature_vector_calculator.eval().cuda()
        Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cuda()
        
        correct_arr = torch.empty(0).cpu()
        for _ in range(iteration):
#             feature_vector = Feature_vector_calculator(points)
            feature_vector = fv[first:last]
            pred, _ = Classifier_from_feature_vector(feature_vector)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.long().data).cpu()
            correct_arr = torch.cat((correct_arr, correct), dim=0)
            
            first = last
    correct_arr = correct_arr.view(iteration, B)#(100, 103)
    correct_arr = correct_arr.detach().numpy().astype(np.int32)
    correct_arr = np.mean(correct_arr, axis=0) #バッチの正答率配列
    prediction_accuracy = np.append(prediction_accuracy, correct_arr)
    
# prediction_accuracy = prediction_accuracy.reshape(2468)
print(prediction_accuracy)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:11<00:00,  7.97s/it]

[1. 1. 1. ... 1. 1. 1.]


In [374]:
print(np.mean(prediction_accuracy))
print(prediction_accuracy[100])
print(np.where(prediction_accuracy == 0))

0.9230145867098866
1.0
(array([ 110,  120,  251,  260,  264,  267,  269,  277,  286,  289,  328,
        385,  409,  468,  469,  479,  607,  620,  650,  695,  698,  711,
        713,  720,  723,  740,  751,  783,  791,  792,  800,  814,  815,
        841,  863,  867,  888,  890,  891,  901,  905,  909,  927,  928,
        938,  940,  943,  945,  946,  947,  948,  949,  950,  951,  952,
        953,  954,  955,  956,  957,  958,  960,  961,  972,  973, 1057,
       1186, 1187, 1240, 1270, 1302, 1303, 1362, 1372, 1377, 1428, 1433,
       1437, 1441, 1442, 1444, 1449, 1454, 1468, 1471, 1473, 1475, 1487,
       1495, 1498, 1503, 1504, 1506, 1511, 1512, 1532, 1533, 1568, 1592,
       1594, 1618, 1629, 1633, 1639, 1645, 1648, 1651, 1655, 1672, 1673,
       1677, 1682, 1687, 1698, 1705, 1706, 1711, 1712, 1713, 1734, 1736,
       1738, 1739, 1743, 1746, 1776, 1789, 1790, 1821, 1853, 1864, 1867,
       1868, 1869, 1923, 1982, 1986, 1990, 1993, 2000, 2001, 2013, 2026,
       2027, 2033, 2044, 20

In [218]:
k = 2
n = 6

print(type(k))
print(type(n))
print(type(n/k))

a = np.random.rand(2, n)
print(a)

idx = create_select_idx(n, int(n/k))
print(idx)

a = np.delete(a, idx, axis=1)
print(a)

a = np.tile(a, k)
print(a)

<class 'int'>
<class 'int'>
<class 'float'>
[[0.95017612 0.55665319 0.91560635 0.64156621 0.39000771 0.48599067]
 [0.60431048 0.54954792 0.92618143 0.91873344 0.39487561 0.96326253]]
[0, 3, 4]
[[0.55665319 0.91560635 0.48599067]
 [0.54954792 0.92618143 0.96326253]]
[[0.55665319 0.91560635 0.48599067 0.55665319 0.91560635 0.48599067]
 [0.54954792 0.92618143 0.96326253 0.54954792 0.92618143 0.96326253]]


In [180]:
a = torch.randn(4, 3)
b = torch.randn(4, 3)

print(a)
print(b)

print(torch.cdist(a, b))
print(torch.sum(torch.cdist(a, b), dim=0))

tensor([[-0.1831,  1.3544,  0.6307],
        [-0.0396, -1.8815,  1.4039],
        [-2.3932,  2.1243, -2.5812],
        [-1.3034,  0.7488, -0.2790]])
tensor([[-0.1542,  0.7707, -0.3421],
        [ 0.0608,  1.1338, -0.9343],
        [-0.3977,  1.5343,  0.0781],
        [ 0.6445,  1.8089, -1.7838]])
tensor([[1.1348, 1.5992, 0.6195, 2.5926],
        [3.1774, 3.8169, 3.6815, 4.9243],
        [3.4437, 3.1169, 3.3767, 3.1564],
        [1.1511, 1.5616, 1.2509, 2.6800]])
tensor([ 8.9070, 10.0946,  8.9286, 13.3533])


In [188]:
a = torch.randint(5, (2, 1, 3)).to(torch.float32)
b = torch.randint(5, (2, 1, 3)).to(torch.float32)

print(a)
print(b)

print(torch.cdist(a, b))
print(torch.sum(torch.cdist(a, b), dim=0))
print(torch.sum(torch.cdist(a, b), dim=0)[0])

tensor([[[1., 0., 1.]],

        [[0., 3., 4.]]])
tensor([[[1., 4., 2.]],

        [[3., 1., 1.]]])
tensor([[[4.1231]],

        [[4.6904]]])
tensor([[8.8135]])
tensor([8.8135])


In [190]:
# feature_vector = torch.empty(len(test_dataset), 1024)
score = torch.empty(0).cuda()
seed = 1
initialize(seed)

with torch.no_grad():
    for i, (points, target_feature_vector) in tqdm(enumerate(GADataLoader), total=len(GADataLoader)):
        B, _, _ = points.shape
        points = points.to(torch.float32).cuda()
        target_feature_vector = target_feature_vector.to(torch.float32).cuda()
        points = points.transpose(2, 1)
        Feature_vector_calculator = Feature_vector_calculator.eval().cuda()

#         feature_vector[i * args.batch_size:i * args.batch_size + B] = Feature_vector_calculator(points)
        feature_vector = Feature_vector_calculator(points)
        dist = torch.sum(torch.cdist(feature_vector.view(B, 1, -1), target_feature_vector.view(B, 1, -1)), dim=0)
        score = torch.cat((score, dist[0]), 0)
        
score = torch.sum(score) / len(GADataLoader.dataset)
print(score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:14<00:00,  8.12s/it]

tensor([183.3536, 204.8997, 202.2536, 196.5194, 189.7121, 213.2008, 198.5647,
        210.3924, 213.6801, 227.3547, 175.5612, 161.3842, 223.1965, 199.2656,
        232.8534, 234.3885, 214.7393, 224.7878, 217.7162, 192.6950, 204.4362,
        226.3793, 218.1501, 190.6359])


In [16]:
a = torch.randn(10)
print(a)

idx = create_select_idx(10, 5)
print(idx)

a = a[idx]
print(a)

a = torch.tile(a, (int(10/5),))
print(a)

tensor([-0.2142, -0.4320, -0.7079, -0.1064, -1.2427, -0.4762, -0.6859, -1.5051,
         0.2550,  1.8042])
[9, 3, 2, 7, 1]
tensor([ 1.8042, -0.1064, -0.7079, -1.5051, -0.4320])
tensor([ 1.8042, -0.1064, -0.7079, -1.5051, -0.4320,  1.8042, -0.1064, -0.7079,
        -1.5051, -0.4320])


In [9]:
# import importlib


# importlib.reload(DataLoaderForGAdownsampling)

  4%|████▊                                                                                                       | 111/2468 [03:34<1:14:33,  1.90s/it]

110 : 6


  5%|█████▎                                                                                                      | 121/2468 [03:53<1:14:28,  1.90s/it]

120 : 10


  5%|█████▌                                                                                                      | 128/2468 [04:06<1:14:20,  1.91s/it]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 2463/2468 [1:18:15<00:09,  1.90s/it]

2462 : 10


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 2465/2468 [1:18:19<00:05,  1.90s/it]

2464 : 4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2468/2468 [1:18:25<00:00,  1.91s/it]


In [102]:
#Fitnessのテスト

data_idx = 198
original_points, target_fv, target = GA_dataset.__getitem__(data_idx)
original_points = torch.from_numpy(original_points.astype(np.float32)).clone().view(1, 1024, 3)
target_fv = torch.from_numpy(target_fv.astype(np.float32)).clone().view(1, -1)
target = torch.from_numpy(np.array(target).astype(np.float32)).clone()

Classifier_from_feature_vector = Classifier_from_feature_vector.eval().cuda()
Feature_vector_calculator = Feature_vector_calculator.eval().cuda()

print(type(original_points))

individual = random.sample(range(1024), 512)
score, acc = get_fitness(individual, original_points, target_fv, target, Feature_vector_calculator, Classifier_from_feature_vector, 100)

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [57]:
def make_path(*a):
    path = '/'.join(a)
    
    return path

def make_prefix(**a):
    prefix = ''
    for key in a.keys():
        prefix += key + '{}_'
    prefix = prefix.format(*tuple(a.values()))
    return prefix

def separate_array(object_arr):
    size = object_arr.shape[0]
    new_arr = np.empty(0)
    for i in range(size):
        new_arr = np.append(new_arr, object_arr[i][0])
        new_arr = np.append(new_arr, object_arr[i][1])
    new_arr = new_arr.reshape(size, 2)
    return new_arr

#出力結果を保存する関数
def save_GA_results(log, hof, *args, **kwargs):
    npoints = 1024
    hof_bit = []
    for individual in hof:
        ind_bit = np.zeros(npoints)

        ind_bit[individual] = 1
        hof_bit.append(ind_bit)

    hof_bit = pd.DataFrame(data=hof_bit)

    pprint(hof_bit)
    
    method_name = make_path(*args) + '/Hall of fame' #何の結果か、どのような加工を加えているか、何個ずつか
    prefix = make_prefix(**kwargs) #よく変えるパラメータ
    ext = ".csv"
    df = hof_bit
    #-------------------------------------
    save_dir = FILE_DIR / Path(method_name) 
    # print(save_dir)
    save_dir.mkdir(exist_ok=True, parents=True)
    timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    # print(timestr)
    file_name = prefix + timestr + ext
    df.to_csv(save_dir/file_name, index=False)
    print("Saved in:", save_dir/file_name)
    slack_notify("Saved in:"+ str(save_dir/file_name))


    #logを保存できる形に変換
    log_df = pd.DataFrame(data=log)
    # pprint(log_df)
    # pprint(log_df[['gen','max']])
    log_df.iloc[:, 2:6] = log_df.iloc[:, 2:6].astype("float32")
#     print(log_df)

    method_name = make_path(*args) + '/log' #何の結果か、どのような加工を加えているか、何個ずつか
    prefix = make_prefix(**kwargs)
    ext = ".csv"
    df = log_df
    #-------------------------------------
    save_dir = FILE_DIR / Path(method_name) 
    # print(save_dir)
    save_dir.mkdir(exist_ok=True, parents=True)
    timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    # print(timestr)
    file_name = prefix + timestr + ext
    df.to_csv(save_dir/file_name, index=False)
    print("Saved in:", save_dir/file_name)
    slack_notify("Saved in:"+ str(save_dir/file_name))
    
    return None

In [35]:
def acc_log(array, num_list):
    if len(array) != np.sum(num_list):
        print("error")
        return None
    dim = len(num_list)
    uarray = []
    for num in num_list:
        uarray.append(list(array[:num]))
        array = array[num:]
    acc_log = np.empty(0)
    for n in uarray:
        acc_log = np.append(acc_log, np.mean(n))
        acc_log = np.append(acc_log, np.std(n))
        acc_log = np.append(acc_log, np.min(n))
        acc_log = np.append(acc_log, np.max(n))
    acc_log = acc_log.reshape(-1, 4)
    acc_log = acc_log.T
    return acc_log